In [ ]:
#loading the google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Making the list of path of videos from the folder we are using.
#Using glob library to make a list of videopaths.

import json
import glob
import numpy as np
import cv2
import copy

list_videos =  glob.glob('/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/*.mp4')

In [ ]:
#saving the local variables in a file.
#the program can be continued from a point even after colab times one out

with open('/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/*.mp4/video_files.txt','w') as file:
  for video_file in list_videos:
    
    file.write('%s\n'%video_file)

In [ ]:
#the file is loaded from the video and the 

with open('/content/gdrive/MyDrive/Deepfake_Amruta_Kirti_CNN_LSTM_tryouts/Dataset/kaggle/train_sample_videos/video_files.txt') as file:
  list_videos = file.readlines()

In [ ]:
#counting the number of frames in the videos
#Only videos having framecount greater then 150 in total are kept.
#videos lesser than 150 frames are too short to be processed upon.
#therefore the videos whith less then 150 frames are deleted.

count_of_frames = []
for video in list_videos:
  cap = cv2.VideoCapture(video)
  if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))<150):
    list_videos.remove(video)
    continue
  count_of_frames.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

In [ ]:
#Displaying all the frame count of all videos in the folder,
#displaying the count of videos that are kept out and the ones having greater then 150 are counted
#finally displaying what is the mean count of the frames are retained in the list of videos

print("frames" , count_of_frames)
print("Total number of videos: " , len(count_of_frames))
print('Average frame per video:',np.mean(count_of_frames))

frames [300, 300, 300, 300, 300, 300, 300, 300, 297, 300, 300, 300, 300, 299, 300, 300, 300, 300, 300, 300, 300, 300, 300, 301, 302, 303, 301, 300, 300, 300, 300, 300, 300, 300, 151, 300, 300, 300, 300, 300, 300, 300, 300, 296, 300, 300, 300, 300, 301, 300, 300, 300, 300, 300, 300, 301, 300, 300, 300, 300, 201, 299, 300, 300, 301, 300, 300, 300, 300, 300, 300, 300, 300, 300, 302, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 301, 300, 300, 300, 300, 151, 300, 300, 300, 302, 306, 300, 300, 300, 300, 298, 298, 300, 300, 300, 300, 301, 201, 300, 300, 300, 300, 300, 300, 303, 300, 201, 300, 300, 300, 300, 306, 300, 300, 300, 301, 300, 300, 300, 300, 300, 298, 300, 300, 300, 300, 301, 300, 300, 300, 300, 300, 300, 300, 301, 300, 300, 300, 300, 300, 300, 151, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 301, 300, 300, 300, 300, 300, 300, 300, 300, 151, 300, 300, 300, 300, 300, 300, 302, 300, 300, 300, 300, 300, 300, 300, 300, 300, 299, 303, 300, 300, 300, 300, 30

In [ ]:
def frame_extract(path):
  s_shot_of_video = cv2.VideoCapture(path) 
  success = 1
  while success:
      truth, image = s_shot_of_video.read()
      if truth:
          yield image

In [ ]:
# process the frames
#the video is divided into stacks of frames
#the face recognition library is used to create cropped frames,
# the cropped frames are bunched and released as a video

!pip3 install face_recognition

import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition
from tqdm.autonotebook import tqdm

def make_cropped_face_videos(path_list,out_dir):
  preprocessed_videos =  glob.glob(out_dir+'*.mp4')
  print("No of videos already present " , len(preprocessed_videos))
  for path in tqdm(path_list):
    path_of_video = os.path.join(out_dir,path.split('/')[-1])
    if_file_exists = glob.glob(path_of_video)
    if(len(if_file_exists) != 0):
      print("File Already exists: " , path_of_video)
      continue
    frames = []
    check_flag = 0
    crop_video = cv2.VideoWriter(path_of_video,cv2.VideoWriter_fourcc('M','J','P','G'), 30, (112,112))
    print(crop_video.__sizeof__())
    for count,frame in enumerate(frame_extract(path)):
      if(count <= 150):
        frames.append(frame)
        if(len(frames) == 4):
          faces = face_recognition.batch_face_locations(frames)
          for i,face in enumerate(faces):
            # print(i,face)
            if(len(face) != 0):
              corner1,corner2,corner3,corner4 = face[0]
            try:
              crop_video.write(cv2.resize(frames[i][corner1:corner2,corner3:corner4t,:],(112,112)))
            except:
              pass
          frames = []
    try:
      del corner1,corner2,corner3,corner4
    except:
      pass
    crop_video.release()
    break

     |████████████████████████████████| 100.1 MB 12 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566185 sha256=fd154945b5a98c9150e15461a397ad7ad8988463b5375d9385b485fbcf79e606
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models


In [ ]:
list_videos

['/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6604.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6265.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6513.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6961.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6172.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6839.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6678.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6219.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6845.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6437.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6390.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6056.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6087.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6720.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6894.mp4',
 '/content

In [ ]:
make_cropped_face_videos(list_videos,'/content/drive/My Drive/Deepfake_Amruta_Kirti_CNN_LSTM_tryouts/kaggle_data')

No of videos already present  0


  0%|          | 0/1999 [00:00<?, ?it/s]

File Already exists:  /content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/Face_only/6604.mp4
File Already exists:  /content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/Face_only/6265.mp4
File Already exists:  /content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/Face_only/6513.mp4
File Already exists:  /content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/Face_only/6961.mp4
File Already exists:  /content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/Face_only/6172.mp4
File Already exists:  /content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/Face_only/6839.mp4
File Already exists:  /content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/Face_only/6678.mp4
File Already exists:  /content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/Face_only/6219.mp4
File Already exists:  /content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/Face_only/6845.mp4
File Already exists:  /content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/Face_only/6437.mp4
File Already exists:  /content/drive/MyDrive/CMPE 295A B/dfd

In [ ]:
video_files

['/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6604.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6265.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6513.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6961.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6172.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6839.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6678.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6219.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6845.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6437.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6390.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6056.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6087.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6720.mp4',
 '/content/drive/MyDrive/CMPE 295A B/dfdc_val/dfdc_val/6894.mp4',
 '/content